In [4]:
from __future__ import print_function
import fitz, sys

def split_pdf(finput):
    doc = fitz.open(finput)
    page = doc.loadPage(0)
    if int(page.MediaBox[2]) > 800.0:
        res = fitz.open() 

        for spage in doc:
            r = spage.rect  
            d = fitz.Rect(spage.CropBoxPosition, spage.CropBoxPosition)  
    
            r1 = fitz.Rect(0, 0, r.width*0.5, r.height)
            r2 = fitz.Rect(r.width*0.5, 0, r.width, r.height)
            rect_list = [r1, r2]
    
            for rx in rect_list:  
                rx += d 
                page = res.newPage(-1, width = rx.width, height = rx.height)
                page.showPDFpage(page.rect, doc, spage.number, clip = rx)

        res.save(doc.name[:-3]+'_sp.pdf', garbage = 3, deflate = True)
        return doc.name[:-3]+'_sp.pdf'
    else:
        return finput

In [5]:
import pdftotext
import os

def text22text(finput):
    
    fout = os.path.join(os.getcwd(), 'output.txt')
    
    if '.pdf' in finput:
         
        with open(finput, "rb") as f:
            pdf = pdftotext.PDF(f)

        with open('output.txt', 'w',-1,'utf-8') as f:
            f.write("\n\n".join(pdf))
    
        with open('output.txt', 'r',-1,'utf-8') as f:
            text=f.read()
            
    elif '.HWP' or '.hwp' in finput:
        hwp2txt_cmd = r'hwp5txt --output={} {}'.format(fout, finput)
        os.system(hwp2txt_cmd)
        with open(fout,'r',encoding='UTF8') as f:
            text=f.read()
            
    return text

In [40]:
import glob
from PIL import Image
#from pytesseract import *
from PyPDF2 import PdfFileReader
from pdf2image import convert_from_bytes
#import cv2
from pathos.multiprocessing import ProcessingPool as Pool

def image22text(finput, seqnum):
    import cv2
    img_name = os.path.join(r'C:\pgm_file', 'img_file', seqnum) 
    if '.hwp' in finput:
        return 0
    
    if not os.path.exists(img_name):
        os.mkdir(img_name)
        images = convert_from_bytes(open(finput, 'rb').read(), dpi=300, output_folder = img_name, fmt="png")
    
    img_list = list()
    for i in range(len(os.listdir(img_name))):
        img_list.append(img_name)
        
    with Pool(10) as p:
        text = p.map(image_multi, img_list, os.listdir(img_name))
    return " ".join(text)

def image_multi(img_name, img):
    import cv2
    import os
    import pytesseract
    print(img_name, img)
    img = cv2.imread(os.path.join(img_name, img))
    img = cv2.resize(img, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 85, 11)

    custom_config = r'-l kor -c tessedit_char_blacklist=ABCDEFGHIJKLMNOPQRSTUVWXYZ --psm 6'
    temp = pytesseract.image_to_string(thresh, config = custom_config, lang='kor')
    return temp

In [41]:
import os
import pandas as pd 

tlist, ilist = list(), list()
xname = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#1_file_list'}.xlsx")
df = pd.read_excel(xname)

for i in range(len(df)):
    
    finput = df.loc[i, 'path']
    print(finput)
    if 'pdf' in finput:
        finput = split_pdf(finput)

    text = text22text(finput)
    if text.isspace():
        text = image22text(finput, str(df.loc[i, 'seqnum']))
        ilist.append('1')
    else:
        ilist.append('0')
    tlist.append(text)
    
    print(i)
df['text'] = tlist
df['image'] = ilist
x2name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#2_text'}.xlsx")
df.to_excel(x2name)

C:\pgm_file\[형사] 특정범죄 가중처벌 등에 관한 법률(2016. 1. 6. 법률 제13717호로 개정된 것) 제5조의4 제5항 제1호 중 형법 제329조 부분에 대하여 위헌법률심판을 제청한 사건(의정부지방법원 2020노985).pdf
0
C:\pgm_file\[형사] 인근 토지 소유자의 원룸 신축공사로 인하여 자신 소유 토지들에 대한 사용권이 침해당한다는 이유로 공사부지 진입로에 철제 펜스를 설치한 피고인에 대하여 일반교통방해죄 및 업무방해죄로 벌금 200만 원을 선고한 사건(2019고정1947).pdf
1
C:\pgm_file\[형사]코로나19 상황을 이용하여 마스크를 팔겠다며 인터넷 사기범행을 저지른 피고인에게 실형을 선고한 사례(서울북부지방법원 2020고단1349).pdf
2
C:\pgm_file\[형사]술에 취해 식당에서 자신이 코로나19에 걸렸다고 소리치며 가래침을 뱉는 등의 행위를 한 피고인에게 집행유예(사회봉사 포함) 등을 선고한 사례(서울북부지방법원 2020고단1347).pdf
3
C:\pgm_file\[형사]아파트 각 층을 돌며 현관문 앞에 놓인 택배상자를 훔친 피고인에게 벌금형을 선고한 사례(서울북부지방법원 2020고단603).pdf
4
C:\pgm_file\[형사]1회 30만 원의 대가를 받고 금융위원회 직원을 사칭하여 보이스피싱 범죄에 가담한 피고인에게 실형을 선고한 사례(서울북부지방법원 2019고단4983).pdf
5
C:\pgm_file\[형사]대출을 돕겠다며 피해자의 신분증과 공인인증서 등을 받은 뒤 피해자 명의로 대출을 받아 이를 가로챈 피고인에게 실형을 선고한 사례(서울북부지방법원 2019고단4726).pdf
6
C:\pgm_file\[형사]재물손괴(전주지방법원 2019고단483).pdf
7
C:\pgm_file\[형사] 산림훼손 실형(울산지방법원 2019고합148).pdf
8
C:\pgm_file\[형사] 마세라티 견인 절도 사건(울산지방법원 2019고단2768).pdf
9
C:\pgm_file\[형사] 소개팅 젓갈,

115
C:\pgm_file\[형사] 위증 사건(울산지방법원 2018고단3690).pdf
116
C:\pgm_file\[형사] 감금 사건(울산지방법원 2019고정365).pdf
117
C:\pgm_file\[형사] 자살방조미수 사건(울산지방법원 2019고합241).pdf


KeyboardInterrupt: 

In [ ]:
from hunspell import Hunspell
h = Hunspell('ko', hunspell_data_dir='./')
h.suggest('변후사')

In [39]:
df.loc[0, 'path']

'C:\\pgm_file\\[형사] 특정범죄 가중처벌 등에 관한 법률(2016. 1. 6. 법률 제13717호로 개정된 것) 제5조의4 제5항 제1호 중 형법 제329조 부분에 대하여 위헌법률심판을 제청한 사건(의정부지방법원 2020노985).pdf'